**Веб-скрейпинг** (или скрепинг, или скрапинг) — это технология получения веб-данных путем извлечения их со страниц веб-ресурсов.  
Веб-скрейпинг объединяет в себе функции краулера (обход сайта и сбор данных) и парсера (анализ содержимого).  
Но чаще для обозначения всего процесса сбора и анализа информации используют слово **парсер**.

### Requests
Python Requests — это библиотека, которая создана для быстрой и простой работы с запросами.

GET - запрашивает представление ресурса. Запросы с использованием этого метода могут только извлекать данные.

HEAD - запрашивает ресурс так же, как и метод GET, но без тела ответа.

POST - используется для отправки сущностей к определённому ресурсу. Часто вызывает изменение состояния или какие-то побочные эффекты на сервере.

PUT - заменяет все текущие представления ресурса данными запроса.

DELETE - удаляет указанный ресурс.

PATCH - используется для частичного изменения ресурса.

In [ ]:
# Установка
!pip install requests

In [ ]:
import requests

In [ ]:
# Ссылка на ресурс для теста запросов
url = "https://httpbin.org/get"

In [ ]:
response = requests.get(url)

In [ ]:
response.text

In [ ]:
"""
HTTP headers позволяют клиенту и серверу передавать дополнительную информацию
с помощью HTTP-запроса или ответа.
'User-Agent' представляет клиентскую программу, инициирующую запрос.
"""
headers = {
    "User-Agent": "Custom Agent",
    "Content-Type": "application/json; charset=utf-8"
}

In [ ]:
# get запрос без headers
requests.get(url).json()

In [ ]:
# get запрос с headers
# в качестве параметров в url передается словарь {"key": "value"}
response = requests.get(url, headers=headers, params={"key": "value"})
print(response.json())
print(response.url)

In [ ]:
# post запрос
data = {'somekey': 'somevalue'}

# Словарь data автоматически кодируется как HTML форма
response = requests.post("https://httpbin.org/post", data=data)
print(response.text)

In [ ]:
# get запрос к официальному сайту Python
python_url = "https://www.python.org"
response = requests.get(python_url)

In [ ]:
# Заголовки ответа
response.headers

In [ ]:
# Содержимое ответа в unicode
response.text

In [ ]:
response.status_code

### Beautiful Soup

BeautifulSoup4 (bs4) - это библиотека Python для извлечения данных из файлов HTML и XML.

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
# get запрос к официальному сайту Python
url = "https://www.python.org"
response = requests.get(url)
response.status_code

Чтобы разобрать HTML-документ, необходимо передать его в конструктор класса **BeautifulSoup()**.  
Можно передать строку или открытый дескриптор файла.

In [ ]:
# Поддерживает несколько видов парсеров: html.parser, lxml, html5lib и xml
bs = BeautifulSoup(response.text, "lxml")
print(bs.prettify())

In [ ]:
# Получаем информацию отдельно по тегам
print(bs.title)
# можно использовать метод text для получения содержимого тега
print(bs.title.text)

Методы поиска тегов:  
**find()** возвращает первый элемент по переданным фильтрующим аргументам.  
**find_all()** просматривает и извлекает ВСЕХ потомков тега, которые соответствуют переданным фильтрующим аргументам.  

**Аргументы:**  
**name** - фильтр по имени HTML-тега, может быть:
 - строкой с именем HTML-тега,
 - списком с несколькими именами HTML-тегов,
 - объектом регулярного выражения,
 - значением bool,
 - функцией.

**attrs** - словарь, с атрибутами HTML-тега в виде {"class": "sister"},  
**recursive** - отвечает за рекурсивный просмотр потомков, по умолчанию True,  
**string** - ищет совпадение в тексте HTML-документа, а не в тегах. Принимаемые значения такие же, как у аргумента name.  
**limit** - целое число, ограничивает максимальное количество совпадений.  
****kwargs** - фильтр по атрибутам HTML-тега. Принимаемые значения такие же, как у аргумента name.  

In [ ]:
# получаем содержимое тега со спсиком новостей
latest_news = bs.find('div', class_='medium-widget blog-widget')

In [ ]:
type(latest_news)

In [ ]:
# далее можно применить find_all для получения списка элементов li
data = [{'date': li.time.text, 'news': li.a.text} for li in latest_news.find_all('li')]
data

In [ ]:
# Для дальнейше обработки полученные данные можно представить в виде DataFrame
import pandas as pd
df = pd.DataFrame(data)
df

In [ ]:
# Запись результата в файл
import csv
with open('news.csv', 'w', newline='', encoding='utf8') as csvfile:
    fieldnames = ['date', 'news']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(data)

### icrawler

**icrawler** предоставляет удобные инструменты для скачивания изображений из различных интернет-ресурсов. Этот модуль поддерживает несколько источников, таких как Google Images, Bing Images, Flickr и другие.

Модуль **icrawler** включает в себя несколько классов, каждый из которых предназначен для работы с определенным источником изображений. Вот несколько ключевых элементов:
1. GoogleImageCrawler
2. BingImageCrawler
3. FlickrImageCrawler
4. GreedyImageCrawler

In [ ]:
!pip install icrawler

In [ ]:
# Импорт нужных классов
from icrawler.builtin import (
    GoogleImageCrawler,
    GreedyImageCrawler
)
import logging

In [ ]:
# Cкачивание 5 изображений из Google Images
print("start testing GoogleImageCrawler")
google_images_dir = "images/google"
google_crawler = GoogleImageCrawler(
    downloader_threads=4, storage={"root_dir": google_images_dir}, log_level=logging.INFO
)
search_filters = dict(size="large", color="orange", license="commercial,modify", date=(None, (2017, 11, 30)))
google_crawler.crawl("cat", filters=search_filters, max_num=5)

In [ ]:
# Cкачивание 5 изображений из заданного по url ресурса
print("start testing GreedyImageCrawler")
greedy_images_dir = "images/greedy"
greedy_crawler = GreedyImageCrawler(parser_threads=4, storage={"root_dir": greedy_images_dir})
greedy_crawler.crawl("https://mixkit.co/", max_num=5, min_size=(100, 100))

#### Полезные ссылки:
Википедия: https://ru.wikipedia.org/wiki/Веб-скрейпинг  
Документация Requests: https://requests.readthedocs.io/en/latest/  
Документация BeautifulSoup4: https://www.crummy.com/software/BeautifulSoup/bs4/doc/  
Документация icrowler: http://icrawler.readthedocs.io/  
HTTP headers: https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers